<a href="https://colab.research.google.com/github/gogela/Colab_COSMOS/blob/main/cosmos_nvidia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Colab NTB for NVIDIA cosmos

In [ ]:
#uninstal default pytorch & cuda
!pip uninstall -y torch torchvision torchaudio

!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove
!apt-get update

Found existing installation: torch 2.5.1+cu121
Uninstalling torch-2.5.1+cu121:
  Successfully uninstalled torch-2.5.1+cu121
Found existing installation: torchvision 0.20.1+cu121
Uninstalling torchvision-0.20.1+cu121:
  Successfully uninstalled torchvision-0.20.1+cu121
Found existing installation: torchaudio 2.5.1+cu121
Uninstalling torchaudio-2.5.1+cu121:
  Successfully uninstalled torchaudio-2.5.1+cu121
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Note, selecting 'nvidia-driver-550-server' for glob 'nvidia*'
Note, selecting 'nvidia-firmware-550-server-550.144.03' for glob 'nvidia*'
Note, selecting 'nvidia-firmware-535-535.154.05' for glob 'nvidia*'
Note, selecting 'nvidia-docker2' for glob 'nvidia*'
Note, selecting 'nvidia-firmware-560-server-560.28.03' for glob 'nvidia*'
Note, selecting 'nvidia-cuda-toolkit-doc' for glob 'nvidia*'
Note, selecting 'nvidia-imex' for glob 'nvidia*'
Note, selecting 'nvidia-dkms-450-server' for glob 'nvi

In [ ]:
#cuda 12.6
!wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64/cuda-keyring_1.1-1_all.deb
!sudo dpkg -i cuda-keyring_1.1-1_all.deb
!sudo apt-get update
!sudo apt-get -y install cuda-toolkit-12-6

--2025-01-23 15:23:10--  https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64/cuda-keyring_1.1-1_all.deb
Resolving developer.download.nvidia.com (developer.download.nvidia.com)... 23.45.46.233, 23.45.46.229
Connecting to developer.download.nvidia.com (developer.download.nvidia.com)|23.45.46.233|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4332 (4.2K) [application/x-deb]
Saving to: ‘cuda-keyring_1.1-1_all.deb’

cuda-keyring_1.1-1_ 100%[===================>]   4.23K  --.-KB/s    in 0s      

2025-01-23 15:23:10 (2.12 GB/s) - ‘cuda-keyring_1.1-1_all.deb’ saved [4332/4332]

Selecting previously unselected package cuda-keyring.
(Reading database ... 122448 files and directories currently installed.)
Preparing to unpack cuda-keyring_1.1-1_all.deb ...
Unpacking cuda-keyring (1.1-1) ...
Setting up cuda-keyring (1.1-1) ...
Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Get:2 https://developer.download.nvidia.com/comp

In [ ]:
!pip uninstall fastai -y
!pip install git+https://github.com/NVlabs/Pytorch_Retinaface.git@b843f45


Found existing installation: fastai 2.7.18
Uninstalling fastai-2.7.18:
  Successfully uninstalled fastai-2.7.18
  Cloning https://github.com/NVlabs/Pytorch_Retinaface.git (to revision b843f45) to /tmp/pip-req-build-b6h8s_pa
  Running command git clone --filter=blob:none --quiet https://github.com/NVlabs/Pytorch_Retinaface.git /tmp/pip-req-build-b6h8s_pa
  Running command git checkout -q b843f45
  Resolved https://github.com/NVlabs/Pytorch_Retinaface.git to commit b843f45
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.5/906.5 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 99.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 76.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.6 MB/s et

In [ ]:
#new pytorch (night build)

#!pip install --pre torch torchvision torchaudio --index-url https://download.pytorch.org/whl/nightly/cu126
!pip install torch==2.7.0.dev20250114+cu126 --index-url https://download.pytorch.org/whl/nightly/cu126
!pip install torchvision==0.22.0.dev20250114+cu126 --index-url https://download.pytorch.org/whl/nightly/cu126



Looking in indexes: https://download.pytorch.org/whl/nightly/cu126
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 82.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.7/897.7 kB 41.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 110.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.0/571.0 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.2/200.2 MB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 38.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.2/158.2 MB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.6/216.6 MB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.8/156.8 MB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.3/89.3 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
#!nvcc --version

# pokus 1  torch 2.7.0.dev20250116+cu126 (17/1/25)
# fastai 2.7.18 requires torch<2.6,>=1.10, but you have torch 2.7.0.dev20250116+cu126 which is incompatible.
# !pip uninstall fastai -y
# install torch again

# pokus 2 - skipping 2.7.0.dev20250115+cu126 - probably didnt work yesterday, so try 2.7.0.dev20250114+cu126


import torch
print(torch.backends.cudnn.version()) #should be at least 90501


90501


In [ ]:
#hf token
import os
from huggingface_hub import login
from google.colab import userdata
os.environ["HF_TOKEN"] = userdata.get("HF_TOKEN")
login(os.environ["HF_TOKEN"])



Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [ ]:
# git
%cd /content/
# 1. Clone the repository and move to the directory (execute only once)
!(if [ ! -d "Cosmos" ]; then \
    git clone https://github.com/NVIDIA/Cosmos.git; \
  fi)
%cd Cosmos


/content
Cloning into 'Cosmos'...
remote: Enumerating objects: 271, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 271 (delta 12), reused 5 (delta 5), pack-reused 254 (from 1)
Receiving objects: 100% (271/271), 9.40 MiB | 34.74 MiB/s, done.
Resolving deltas: 100% (53/53), done.
/content/Cosmos


In [ ]:
#!patch /content/Cosmos/cosmos1/models/diffusion/module/pretrained_vae.py
!cp /content/pretrained_vae.py /content/Cosmos/cosmos1/models/diffusion/module/pretrained_vae.py

In [ ]:
# upload sample video
!cp /content/sample_water.mp4 /content/sample.mp4

In [ ]:
%cd /content/Cosmos

# Not sure this is even needed
!apt-get update && apt-get install -y ffmpeg
!pip install --no-cache-dir -r requirements.txt
!pip install pynvml
#!pip install transformer_engine[pytorch]




/content/Cosmos
Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree

In [ ]:
!pip uninstall transformer_engine
!pip cache purge
!pip install transformer_engine[pytorch]

Files removed: 122
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.4/121.4 kB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 MB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 460.0/460.0 kB 37.6 MB/s eta 0:00:00
  Created wheel for transformer_engine_torch: filename=transformer_engine_torch-1.13.0-cp311-cp311-linux_x86_64.whl size=30541637 sha256=d8ea033703c2e1830ff83b268eb7006bd5e02fd76a12ff7a24aa483e82ca48c6
  Stored in directory: /root/.cache/pip/wheels/0a/a4/a9/ffe65f45da165ca92fbd32267de0cbf654dc56221e3105af54
Successfully built transformer_engine_torch


Diffusion

In [ ]:
#download models - diffusion
#redirect uoutput to /dev/null - otherwise the lenghty output kills browser
!PYTHONPATH=$(pwd) python cosmos1/scripts/download_diffusion.py --model_sizes 7B --model_types Video2World >/tmp/download.log 2>&1
#!PYTHONPATH=$(pwd) python cosmos1/scripts/download_diffusion.py --model_sizes 7B 14B --model_types Text2World Video2World >/tmp/download.log 2>&1


In [ ]:
#Inference - diffusion
!(export PYTHONPATH=$(pwd) && \
  python cosmos1/models/diffusion/inference/video2world.py \
    --checkpoint_dir checkpoints \
    --diffusion_transformer_dir Cosmos-1.0-Diffusion-7B-Video2World \
    --prompt 'Pouring a glass of water in a kitchen' \
    --input_image_or_video_path /content/sample.mp4 \
    --num_input_frames 1 \
    --disable_prompt_upsampler \
    --video_save_folder video_output \
    --video_save_name Cosmos-1.0-Diffusion-7B-Video2World \
    --offload_diffusion_transformer \
    --offload_tokenizer \
    --offload_text_encoder_model \
    --offload_prompt_upsampler \
    --offload_guardrail_models)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.
0it [00:00, ?it/s]
2025-01-23 16:21:03.026034: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-23 16:21:03.674902: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-23 16:21:03.966519: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-23 16:21:04.053343: E exte

Autoregression

In [ ]:
#Inference - autoregressive
%cd /content/Cosmos
!PYTHONPATH=$(pwd) python cosmos1/scripts/download_autoregressive.py --model_sizes 4B 5B >/tmp/download.log 2>&1

/content/Cosmos


In [ ]:
%cd /content/Cosmos
!CUDA_VISIBLE_DEVICES=0 PYTHONPATH=$(pwd) python cosmos1/models/autoregressive/inference/video2world.py \
    --input_type=text_and_video \
    --input_image_or_video_path=/content/sample.mp4  \
    --prompt="A video showing a kitchen sink and a tap. A hand holds a cup and water from the tap fills the cup" \
    --video_save_name=Cosmos-1.0-Autoregressive-5B-Video2World \
    --ar_model_dir=Cosmos-1.0-Autoregressive-5B-Video2World \
    --top_p=0.7 \
    --temperature=1.0 \
    --offload_guardrail_models \
    --offload_diffusion_decoder \
    --offload_ar_model \
    --offload_tokenizer \
    --offload_text_encoder_model

/content/Cosmos
2025-01-17 16:22:08.377728: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-17 16:22:08.397951: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-17 16:22:08.419627: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-17 16:22:08.426210: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-17 16:22:08.442387: I tensorflow/core/platform/